In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
"""
Notebook for YOLO11s training
"""

In [ ]:
if torch.cuda.is_available():
    device = 0
    print(f"Training on GPU")
else:
    print(f"Training on CPU")
    device = "cpu"

In [ ]:
"""
Default values used for all YOLO trainings.

These parameters should remain consistent across different training runs,
unless the hardware or training context changes significantly.
"""

model = YOLO("yolo11s.pt")  # pretrained YOLOv11L model

default_args = {
    "data": "config/materials.yaml",     # dataset configuration path
    "project": "material_koulutus",      # output base folder
    "imgsz": 768,                     # default input size for YOLO
    "batch": 16,                      # maximum batch size (GPU memory limit)
    "device": 0,                      # GPU device ID
    "cache": "disk",                   # cache data in RAM for speed
    "workers": 4,                     # parallel data loader workers
    "amp": True,                      # automatic mixed precision (faster training)
    "close_mosaic": 10,               # disable mosaic in last N epochs
    "save": True,                     # save checkpoints and weights
    "val": True,                      # enable validation after each epoch
    "plots": True                     # save training/validation plots
}

In [ ]:
"""
Run-specific training parameters.

These parameters can be adjusted per experiment to test different configurations,
optimizers, learning rates, or other hyperparameters.
"""

results = model.train(
    **default_args,
    epochs=100,                        # number of training epochs
    optimizer="AdamW",                # optimizer: AdamW, Adam, or SGD
    lr0=0.0007,                        # initial learning rate
    momentum=0.85,                     # used with optimizers like SGD or AdamW
    weight_decay=0.0003,              # regularization to prevent overfitting
    warmup_epochs=3.0,                # warm-up period for learning rate
    name="train_mika_11s_e100_i768_run"             # unique name for this training run
)

In [ ]:
"""
This script reads YOLO training results from a CSV file and visualizes key evaluation metrics:
- Training and validation losses
"""

%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

csv_path = "material_koulutus/train_minna_v3/results.csv"
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()

display(df.head())
display(df.tail())

df[["train/box_loss", "train/cls_loss", "train/dfl_loss"]].plot(figsize=(10, 5), title="Training Losses per Epoch")
plt.xlabel("Epoch")
plt.grid()
plt.show()